In [2]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

### 多输入通道

当有$c$个channel时，对于单张图像，kernel的size变为：
$$c\times k_h \times k_w $$

在各个channel上对输入的二维数组和卷积核的二维核数组做互相关运算，再将$c$个互相关运算的二维输出按通道相加，得到最终的二维输出结果。

In [6]:
import torch 
from torch import nn
import sys
import dl_5_1 as d2l

importing Jupyter notebook from dl_5_1.ipynb
Step 5, loss 0.146
Step 10, loss 0.019
Step 15, loss 0.003
Step 20, loss 0.001
weight:  tensor([[ 1.0069, -1.0031]])
bias:  tensor([-0.0021])


In [7]:
def corr2d_multi_in(X,K):
    res = d2l.corr2d(X[0,:,:],K[0,:,:])
    for i in range(1,X.shape[0]):
        res+=d2l.corr2d(X[i,:,:],K[i,:,:])
    return res

In [22]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

In [23]:
X.shape #CHW

torch.Size([2, 3, 3])

In [24]:
K.shape

torch.Size([2, 2, 2])

In [25]:
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 多输出通道

设输入通道个数为$c_i$，输出通道为$c_o$，核数组的高宽为$k_h$，$k_w$。如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i \times k_h \times k_w$的核数组，然后该输出通道上的结果由互相关运算输出相加。那么对于$c_o$个输出通道，创建$c_o$个核数组。因此整个核数组维度可以看做：
$$c_o \times c_i \times k_h \times k_w$$

In [26]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K])

In [27]:
K=torch.stack([K,K+1,K+2])
K.shape

torch.Size([3, 2, 2, 2])

In [28]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

#### $1\times 1$卷积层

In [35]:
def corr2d_multi_in_out_1x1(X,K):
    c_i, h , w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i,h*w)
    K = K.view(c_o,c_i)#kernel [c_o, c_ 1, 1 , 1]
    Y = torch.mm(K,X)#全连接层
    return Y.view(c_o,h,w)

In [31]:
X= torch.rand(3,3,3)
K = torch.rand(2,3,1,1)

In [32]:
Y1 = corr2d_multi_in_out_1x1(X,K)

In [33]:
Y2 = corr2d_multi_in_out(X,K)

这里Y1相当于全连接层，Y2相当于卷积（1x1），二者保持了一致。

In [34]:
(Y1-Y2).norm().item() < 1e-6

True